In [1]:
import os 
import numpy as np
import re
from dtw import dtw
from tqdm.notebook import tqdm
print("this is test file!")
import seaborn as sns 
from sklearn.cluster import SpectralClustering
import pickle
# np.set_printoptions(threshold=np.inf)
sns.set(color_codes=True)

this is test file!


## 1.根据相似矩阵进行谱聚类的计算 

In [2]:
def normalized_matrix(data):
    data_min = np.min(data.flatten())
    data_max = np.max(data.flatten())
    nor_data = (data-data_min) /(data_max-data_min)
    return nor_data

path = "./processing/DTW_similarity_matrix.npy"
DTW_similarity = np.load(path, allow_pickle=True)
# print(DTW_similarity)
nor_data = normalized_matrix(DTW_similarity)
print(nor_data.shape)
DTW_similarity = nor_data
# print(nor_data)


(4224, 4224)


### 1.对相似度进行正则化


In [3]:
# 根据邻接矩阵计算谱聚类的结果

save_path = "./processing/DTW_similarity_matrix_clustering.pkl"

cluster_result = {}
rang_clustering = (2, 100)
for i in tqdm(range(rang_clustering[0], rang_clustering[1]+1)):
    n_clusters = i
    seeg_clustering = SpectralClustering(n_clusters=n_clusters, assign_labels="discretize", random_state=0, affinity = "precomputed").fit_predict(DTW_similarity)
#     print(seeg_clustering)
    id_key = "cluster_{}".format(i)
    cluster_result[id_key] = seeg_clustering

with open(save_path, 'wb') as f:
    pickle.dump(cluster_result, f)
    print("Saving result: {}".format(save_path))
    


Saving result: ./processing/DTW_similarity_matrix_clustering.pkl


## 2. 根据谱聚类的结果进行聚类效果的分析

In [4]:
# print(cluster_result['cluster_7'])
cluster_result = np.load("./processing/DTW_similarity_matrix_clustering.pkl", allow_pickle=True)
path_data = "./data/BDP_val_prediction.pkl"
feature_val_prediction = np.load(path_data, allow_pickle=True)
print(feature_val_prediction)
count_ground = [0,0]
count_pre = [0,0]
filter_feature_val_prediction ={}
for id, d in feature_val_prediction.items():
    count_ground[d["ground truth"]] += 1
    count_pre[d["prediction"]] += 1 
    if d["ground truth"] == d["prediction"]:
        filter_feature_val_prediction[id] = d
print("原本的数据分布！")
print("ground_truth:{}  prediction:{}".format(count_ground, count_pre))

count_ground = [0,0]
count_pre = [0,0]
print("预测结果的数据分布！")
for id, d in filter_feature_val_prediction.items():
    count_ground[d["ground truth"]] += 1
    count_pre[d["prediction"]] += 1                    
print("ground_truth:{}  prediction:{}".format(count_ground, count_pre))

{'a5531f6d-fa3a-11e9-a8f2-e0d55e6ff654-2.npy': {'ground truth': 0, 'prediction': 1}, 'a5c1d8f1-fa3a-11e9-a8f2-e0d55e6ff654-2.npy': {'ground truth': 0, 'prediction': 0}, 'a5c1d8f4-fa3a-11e9-a8f2-e0d55e6ff654-2.npy': {'ground truth': 0, 'prediction': 1}, 'a4e5e672-fa3a-11e9-a8f2-e0d55e6ff654-2.npy': {'ground truth': 0, 'prediction': 0}, 'a59dcf43-fa3a-11e9-a8f2-e0d55e6ff654-2.npy': {'ground truth': 0, 'prediction': 1}, 'b1547040-2555-11ea-9699-e0d55e6ff654-0.npy': {'ground truth': 1, 'prediction': 0}, 'b1546e37-2555-11ea-9699-e0d55e6ff654-0.npy': {'ground truth': 1, 'prediction': 0}, 'b1546f41-2555-11ea-9699-e0d55e6ff654-0.npy': {'ground truth': 1, 'prediction': 0}, 'b12e6658-2555-11ea-9699-e0d55e6ff654-0.npy': {'ground truth': 1, 'prediction': 1}, 'b1546f3e-2555-11ea-9699-e0d55e6ff654-0.npy': {'ground truth': 1, 'prediction': 0}, 'e076f408-2552-11ea-9699-e0d55e6ff654-0.npy': {'ground truth': 1, 'prediction': 1}, 'b12e67be-2555-11ea-9699-e0d55e6ff654-0.npy': {'ground truth': 1, 'predicti

In [5]:
# print(len(filter_feature_val_prediction))
# print(filter_feature_val_prediction)

In [6]:
from collections import Counter

for id, data in cluster_result.items():
    cluster_data = data
    print(dict(Counter(cluster_data)))
    

{0: 4206, 1: 18}
{0: 4193, 2: 23, 1: 8}
{0: 4177, 1: 11, 3: 28, 2: 8}
{0: 4176, 1: 10, 2: 23, 4: 8, 3: 7}
{0: 4058, 2: 39, 1: 10, 4: 98, 5: 14, 3: 5}
{0: 3867, 1: 191, 3: 54, 4: 10, 6: 83, 5: 14, 2: 5}
{0: 3679, 5: 68, 7: 189, 2: 211, 4: 10, 1: 13, 6: 49, 3: 5}
{0: 3692, 2: 205, 4: 50, 1: 161, 3: 10, 7: 88, 8: 13, 5: 5}
{0: 3693, 3: 143, 2: 187, 6: 10, 8: 92, 4: 74, 1: 11, 5: 9, 7: 5}
{10: 3596, 2: 199, 0: 159, 9: 86, 6: 76, 7: 73, 1: 7, 4: 13, 8: 10, 3: 5}
{0: 3480, 4: 201, 9: 98, 8: 143, 1: 83, 3: 85, 11: 100, 7: 7, 2: 12, 6: 10, 5: 5}
{0: 3495, 3: 131, 1: 191, 2: 155, 6: 75, 10: 65, 11: 7, 9: 71, 4: 7, 7: 12, 12: 11, 5: 4}
{0: 3483, 6: 169, 5: 70, 12: 6, 1: 97, 13: 58, 11: 106, 7: 72, 8: 102, 2: 7, 10: 10, 4: 9, 3: 31, 9: 4}
{0: 3477, 8: 175, 4: 77, 13: 74, 5: 81, 1: 129, 3: 7, 7: 105, 6: 64, 2: 7, 10: 10, 14: 9, 11: 5, 9: 4}
{0: 3459, 10: 131, 7: 101, 6: 95, 2: 162, 15: 77, 13: 88, 12: 6, 3: 68, 5: 7, 1: 10, 9: 9, 11: 4, 4: 7}
{0: 3445, 12: 163, 10: 89, 5: 142, 8: 85, 4: 86, 15: 7,

In [7]:
from sklearn.metrics import davies_bouldin_score
path_data = "./data/feature_true_id_prediction.pkl"
feature_true_id_prediction = np.load(path_data, allow_pickle=True)
feature_true_id_prediction_numpy = []
for id, d in feature_true_id_prediction.items():
    feature_true_id_prediction_numpy.append(d.tolist())
# print(feature_true_id_prediction_numpy)
for n_clusters, d in cluster_result.items(): 
    labels = d
    score = davies_bouldin_score(feature_true_id_prediction_numpy, labels)
    print("{}:{}".format(n_clusters, score))

cluster_2:2.4596500700120703
cluster_3:2.0603312374668095
cluster_4:1.8747850618732413
cluster_5:2.0618565595492058
cluster_6:1.8808215750557373
cluster_7:1.7201862209953467
cluster_8:1.7535955977055315
cluster_9:1.7317162832919104
cluster_10:2.1147240616323555
cluster_11:2.2865778242178854
cluster_12:2.2466590032572378
cluster_13:2.278090564750471
cluster_14:2.203654279829203
cluster_15:2.109681330139323
cluster_16:2.2300178683982352
cluster_17:2.1809508581625456
cluster_18:2.2528070778551057
cluster_19:2.3520661951118256
cluster_20:2.209747261929558
cluster_21:2.4875078368519983
cluster_22:2.5898165225130447
cluster_23:2.4573457062476285
cluster_24:2.3384054606968796
cluster_25:3.2140702298133217
cluster_26:2.376238663155774
cluster_27:2.533074502987449
cluster_28:2.672577166335448
cluster_29:2.5481875696881455
cluster_30:2.773728332496172
cluster_31:2.497572777764396
cluster_32:2.6093727951364407
cluster_33:2.67371163527972
cluster_34:2.7234309013596105
cluster_35:2.565677278304774


### 2.1 选择表现性能最好的簇

In [8]:
# 根据评分选择最好的簇 DBI 得分最低的簇为 cluster=7
cluster_name = "cluster_7"
label_list = cluster_result["cluster_7"]
wave_clusters = {}
wave_clusters_id = {}

# clusters feeature {cluster id :(raw data, raw data id)}
for i ,(id, d)  in enumerate(feature_true_id_prediction.items()):
    label = label_list[i]
    if label not in wave_clusters.keys():
        wave_clusters[label] = []
        wave_clusters_id[label] = []
    wave_clusters[label].append((d, id))
    wave_clusters_id[label].append(id)
# print(wave_clusters)
with open("./processing/wave_clusters_id_feature_data.pkl", 'wb') as f:
    pickle.dump(wave_clusters, f)
    print("Cluster ID-feature data finished!")
with open("./processing/Best_label_cluster.pkl", 'wb') as f:
    pickle.dump(wave_clusters_id, f)
    print("簇滤波id序列构建完成！")

Cluster ID-feature data finished!
簇滤波id序列构建完成！


In [9]:
def similarity_DTW(s1, s2):
    ratio = 50 # 设定的放缩系数，避免数据的相似度过于集中
    euclidean_norm = lambda x, y: np.abs(ratio * (x - y))
    d, cost_matrix, acc_cost_matrix, path = dtw(s1, s2, dist=euclidean_norm)
    score = 1 - np.tanh(d)  # 相似度的评分【0,1】 0： 完全不同， 1： 完全相同
    return score

def get_top_score_series(series_dict, top_n = 5):
    scores = []
    raw_data_series = [x[0] for x in series_dict]
    for s_a, feature_id in tqdm(series_dict):
        score = 0
        for s_b in raw_data_series:
            score += similarity_DTW(s_a, s_b)
        avg_score = score / len(series_dict)
        _ = (feature_id, score)
        scores.append(_)
    result = sorted(scores, key=lambda x:-x[1])
    result = result[:top_n]
    return result

In [10]:
# DBA
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
def sum_of_squares(s,series,cost_mat,delta_mat):
    return sum(map(lambda t:squared_DTW(s,t,cost_mat,delta_mat),series))

def approximate_medoid_index(series,cost_mat,delta_mat):
    if len(series)<=50:
        indices = range(0,len(series))
    else:
        indices = np.random.choice(range(0,len(series)),50,replace=False)

    medoid_ind = -1
    best_ss = 1e20
    for index_candidate in indices:
        candidate = series[index_candidate]
        ss = sum_of_squares(candidate,series,cost_mat,delta_mat)
        if(medoid_ind==-1 or ss<best_ss):
            best_ss = ss
            medoid_ind = index_candidate
    return medoid_ind

def squared_DTW(s,t,cost_mat,delta_mat):
    s_len = len(s)
    t_len = len(t)
    length = len(s)
    fill_delta_mat_dtw(s, t, delta_mat)
    cost_mat[0, 0] = delta_mat[0, 0]
    for i in range(1, s_len):
        cost_mat[i, 0] = cost_mat[i-1, 0]+delta_mat[i, 0]

    for j in range(1, t_len):
        cost_mat[0, j] = cost_mat[0, j-1]+delta_mat[0, j]

    for i in range(1, s_len):
        for j in range(1, t_len):
            diag,left,top =cost_mat[i-1, j-1], cost_mat[i, j-1], cost_mat[i-1, j]
            if(diag <=left):
                if(diag<=top):
                    res = diag
                else:
                    res = top
            else:
                if(left<=top):
                    res = left
                else:
                    res = top
            cost_mat[i, j] = res+delta_mat[i, j]
    return cost_mat[s_len-1,t_len-1]

def fill_delta_mat_dtw(center, s, delta_mat):
    slim = delta_mat[:len(center),:len(s)]
    np.subtract.outer(center, s,out=slim)
    np.square(slim, out=slim)

def DBA_update(center, series, cost_mat, path_mat, delta_mat):
    options_argmin = [(-1, -1), (0, -1), (-1, 0)]
    updated_center = np.zeros(center.shape)
    n_elements = np.array(np.zeros(center.shape), dtype=int)
    center_length = len(center)
    for s in series:
        s_len = len(s)
        fill_delta_mat_dtw(center, s, delta_mat)
        cost_mat[0, 0] = delta_mat[0, 0]
        path_mat[0, 0] = -1

        for i in range(1, center_length):
            cost_mat[i, 0] = cost_mat[i-1, 0]+delta_mat[i, 0]
            path_mat[i, 0] = 2

        for j in range(1, s_len):
            cost_mat[0, j] = cost_mat[0, j-1]+delta_mat[0, j]
            path_mat[0, j] = 1

        for i in range(1, center_length):
            for j in range(1, s_len):
                diag,left,top =cost_mat[i-1, j-1], cost_mat[i, j-1], cost_mat[i-1, j]
                if(diag <=left):
                    if(diag<=top):
                        res = diag
                        path_mat[i,j] = 0
                    else:
                        res = top
                        path_mat[i,j] = 2
                else:
                    if(left<=top):
                        res = left
                        path_mat[i,j] = 1
                    else:
                        res = top
                        path_mat[i,j] = 2

                cost_mat[i, j] = res+delta_mat[i, j]

        i = center_length-1
        j = s_len-1

        while(path_mat[i, j] != -1):
            updated_center[i] += s[j]
            n_elements[i] += 1
            move = options_argmin[path_mat[i, j]]
            i += move[0]
            j += move[1]
        assert(i == 0 and j == 0)
        updated_center[i] += s[j]
        n_elements[i] += 1

    return np.divide(updated_center, n_elements)
def performDBA(series, n_iterations=5):
    n_series = len(series)
    max_length = reduce(max, map(len, series))

    cost_mat = np.zeros((max_length, max_length))
    delta_mat = np.zeros((max_length, max_length))
    path_mat = np.zeros((max_length, max_length), dtype=np.int8)

    medoid_ind = approximate_medoid_index(series,cost_mat,delta_mat)
    center = series[medoid_ind]

    for i in range(0,n_iterations):
        center = DBA_update(center,
                            series, cost_mat, path_mat, delta_mat)

    return center 

In [11]:
# 1.Clustering HeatMap
DBA_id_wave = {}
for cluster_id, data in tqdm(wave_clusters.items()):
    series = []
    for d, id in data:
        series.append(d)
    s_DBA = performDBA(series)
    DBA_id_wave[cluster_id] = s_DBA

with open("./processing/DBA_wave.pkl", 'wb') as f:
    pickle.dump(DBA_id_wave, f)
    print("Saving Success!")


Saving Success!
